# Experiments Management
This notebook shows how to use the [experiment_management.py](https://github.com/abreukuse/ml_utilities/blob/master/experiments_management.py) module. You can use the function `experiment_manager` in the module as a way to track and record machine learning experiments results with the help of [mlflow](https://mlflow.org/docs/latest/index.html).<br>
It is possible to try several configurations of hyperparameters. Also, it is possible to choose between a simple split in the data or cross-validation in both tasks: classification and regression.

In [1]:
# requirements:

# feature_engine==1.0.2
# scikit-learn==0.24.1
# numpy==1.19.3
# pandas==1.2.3

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer, load_diabetes
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, roc_auc_score, log_loss
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, mean_squared_log_error
from experiments_management import experiment_manager

## Classification

In [3]:
data = load_breast_cancer()

In [4]:
X = pd.DataFrame(data.data, columns=data.feature_names)
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
y = pd.Series(data.target)
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int32

### One Experiment

In [6]:
RANDOM_STATE = 77
TEST_SIZE = 0.3

In [7]:
pipeline_logistic_regression = make_pipeline(SklearnTransformerWrapper(transformer=StandardScaler()),
                                             LogisticRegression())

In [8]:
# Setting the hyperparameters search space for logistic regression

def runs_logistic_regression():
    choose = np.random.choice

    C = choose([0.001,0.01,0.1,1,10,100,1000])
    max_iter = choose(range(3000, 10500, 1000))
    solver = choose(['lbfgs', 'sag', 'saga'])
    transformer = choose([StandardScaler(), MinMaxScaler()])

    hyperparameters = {'logisticregression__C': C,
                       'logisticregression__max_iter': max_iter,
                       'logisticregression__solver': solver,
                       'sklearntransformerwrapper__transformer': transformer,
                       'logisticregression__random_state': RANDOM_STATE}
    
    return hyperparameters

In [9]:
metrics = {'precision': precision_score,
           'recall': recall_score,
           'f1_score': f1_score,
           'accuracy': accuracy_score,
           'auc': roc_auc_score,
           'log_loss': log_loss}

#### Simple Split

In [10]:
experiment_manager(task='classification',
                   pipeline=pipeline_logistic_regression, 
                   X=X, 
                   y=y, 
                   runs=3,
                   validation='simple_split',
                   hyperparameters=runs_logistic_regression,
                   metrics=metrics,
                   random_state=RANDOM_STATE,
                   test_size=TEST_SIZE)

Experiment Name: LogisticRegression
Experiment_id: 1

Run 1

HYPERPARAMETERS
C: 1000.0
max_iter: 10000
solver: saga
transformer: MinMaxScaler()
random_state: 77

SCORES
train_precision: 0.713
test_precision: 0.727
train_recall: 1.0
test_recall: 1.0
train_f1_score: 0.832
test_f1_score: 0.842
train_accuracy: 0.749
test_accuracy: 0.76
train_auc: 0.984
test_auc: 0.988
train_log_loss: 0.523
test_log_loss: 0.513

Run 2

HYPERPARAMETERS
C: 10.0
max_iter: 9000
solver: saga
transformer: StandardScaler()
random_state: 77

SCORES
train_precision: 0.623
test_precision: 0.637
train_recall: 1.0
test_recall: 1.0
train_f1_score: 0.768
test_f1_score: 0.779
train_accuracy: 0.623
test_accuracy: 0.637
train_auc: 0.983
test_auc: 0.988
train_log_loss: 0.643
test_log_loss: 0.635

Run 3

HYPERPARAMETERS
C: 1000.0
max_iter: 7000
solver: saga
transformer: MinMaxScaler()
random_state: 77

SCORES
train_precision: 0.929
test_precision: 0.932
train_recall: 1.0
test_recall: 1.0
train_f1_score: 0.963
test_f1_score: 0

#### Cross Validation

In [11]:
experiment_manager(task='classification',
                   pipeline=pipeline_logistic_regression, 
                   X=X, 
                   y=y, 
                   runs=3,
                   validation='cross_validation',
                   hyperparameters=runs_logistic_regression,
                   metrics=metrics,
                   random_state=RANDOM_STATE,
                   cv_method=KFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE),
                   n_splits=2)

Experiment Name: LogisticRegression
Experiment_id: 1

Run 1

HYPERPARAMETERS
C: 0.001
max_iter: 7000
solver: saga
transformer: MinMaxScaler()
random_state: 77

SCORES
train_precision: 0.992
test_precision: 0.969
train_recall: 1.0
test_recall: 0.966
train_f1_score: 0.996
test_f1_score: 0.968
train_accuracy: 0.995
test_accuracy: 0.96
train_auc: 1.0
test_auc: 0.992
train_log_loss: 0.024
test_log_loss: 0.136

Run 2

HYPERPARAMETERS
C: 1.0
max_iter: 8000
solver: lbfgs
transformer: MinMaxScaler()
random_state: 77

SCORES
train_precision: 0.989
test_precision: 0.969
train_recall: 1.0
test_recall: 0.966
train_f1_score: 0.994
test_f1_score: 0.968
train_accuracy: 0.993
test_accuracy: 0.96
train_auc: 1.0
test_auc: 0.991
train_log_loss: 0.028
test_log_loss: 0.125

Run 3

HYPERPARAMETERS
C: 100.0
max_iter: 6000
solver: lbfgs
transformer: StandardScaler()
random_state: 77

SCORES
train_precision: 0.975
test_precision: 0.962
train_recall: 0.997
test_recall: 0.994
train_f1_score: 0.986
test_f1_score: 

### More than one experiment

In [12]:
# I will add the decision tree classifier and run together with the logistic regression
pipeline_decision_tree = make_pipeline(DecisionTreeClassifier())

In [13]:
# Search space for decision trees

def runs_decision_tree():
    choose = np.random.choice

    max_depth = choose(range(2, 16, 2))
    min_samples_split  = choose([5,8,10,12,15,20])
    min_samples_leaf = choose([5,8,10,12,15,20])
    max_features = choose(np.arange(0.1,1.1,0.1))
    min_impurity_decrease = choose(np.arange(0.1,0.6,0.1))
    class_weight = choose([None, 'balanced'])

    hyperparameters = {'decisiontreeclassifier__max_depth': max_depth,
                       'decisiontreeclassifier__min_samples_split': min_samples_split,
                       'decisiontreeclassifier__min_samples_leaf': min_samples_leaf,
                       'decisiontreeclassifier__max_features': max_features,
                       'decisiontreeclassifier__min_impurity_decrease': min_impurity_decrease,
                       'decisiontreeclassifier__class_weight': class_weight,
                       'decisiontreeclassifier__random_state': RANDOM_STATE}
    
    return hyperparameters

In [14]:
metrics = {'precision': precision_score,
           'recall': recall_score,
           'f1_score': f1_score,
           'accuracy': accuracy_score,
           'auc': roc_auc_score,
           'log_loss': log_loss}

In [15]:
# Run both experiments: logistic regression and decision trees

experiments = ((pipeline_logistic_regression, runs_logistic_regression),
               (pipeline_decision_tree, runs_decision_tree))

In [16]:
for pipeline, hyperparameters in experiments:
    experiment_manager(task='classification',
                       pipeline=pipeline, X=X, y=y, 
                       runs=5,
                       validation='simple_split',
                       hyperparameters=hyperparameters,
                       metrics=metrics,
                       random_state=RANDOM_STATE,
                       test_size=TEST_SIZE)

Experiment Name: LogisticRegression
Experiment_id: 1

Run 1

HYPERPARAMETERS
C: 1.0
max_iter: 9000
solver: lbfgs
transformer: StandardScaler()
random_state: 77

SCORES
train_precision: 0.992
test_precision: 0.982
train_recall: 0.996
test_recall: 0.982
train_f1_score: 0.994
test_f1_score: 0.982
train_accuracy: 0.992
test_accuracy: 0.977
train_auc: 0.999
test_auc: 0.998
train_log_loss: 0.042
test_log_loss: 0.054

Run 2

HYPERPARAMETERS
C: 0.01
max_iter: 8000
solver: saga
transformer: MinMaxScaler()
random_state: 77

SCORES
train_precision: 0.912
test_precision: 0.923
train_recall: 1.0
test_recall: 0.991
train_f1_score: 0.954
test_f1_score: 0.956
train_accuracy: 0.94
test_accuracy: 0.942
train_auc: 0.989
test_auc: 0.99
train_log_loss: 0.284
test_log_loss: 0.272

Run 3

HYPERPARAMETERS
C: 1.0
max_iter: 10000
solver: lbfgs
transformer: MinMaxScaler()
random_state: 77

SCORES
train_precision: 0.992
test_precision: 0.982
train_recall: 0.996
test_recall: 0.982
train_f1_score: 0.994
test_f1_sco

## Regression

In [17]:
data = load_diabetes()

In [18]:
X = pd.DataFrame(data.data, columns=data.feature_names)
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [19]:
y = pd.Series(data.target)
y.head()

0    151.0
1     75.0
2    141.0
3    206.0
4    135.0
dtype: float64

In [20]:
ridge = make_pipeline(Ridge())
decision_trees = make_pipeline(DecisionTreeRegressor())

In [21]:
# hyperparameter space:

# ridge
def runs_ridge():
    choose = np.random.choice

    alpha = choose([0.01,0.1,1,10,100])
    max_iter = choose(range(1000, 10500, 1000))
    solver = choose(['cholesky', 'sag', 'saga', 'svd', 'auto'])

    hyperparameters = {'ridge__alpha': alpha,
                       'ridge__max_iter': max_iter,
                       'ridge__solver': solver,
                       'ridge__random_state': RANDOM_STATE}
    
    return hyperparameters

# decision tree
def runs_decision_tree():
    choose = np.random.choice

    max_depth = choose(range(2, 16, 2))
    min_samples_split  = choose([5,8,10,12,15,20])
    min_samples_leaf = choose([5,8,10,12,15,20])
    max_features = choose(np.arange(0.1,1.1,0.1))
    min_impurity_decrease = choose(np.arange(0.1,0.6,0.1))

    hyperparameters = {'decisiontreeregressor__max_depth': max_depth,
                       'decisiontreeregressor__min_samples_split': min_samples_split,
                       'decisiontreeregressor__min_samples_leaf': min_samples_leaf,
                       'decisiontreeregressor__max_features': max_features,
                       'decisiontreeregressor__min_impurity_decrease': min_impurity_decrease,
                       'decisiontreeregressor__random_state': RANDOM_STATE}
    
    return hyperparameters

In [22]:
experiments = ((ridge, runs_ridge),
               (decision_trees, runs_decision_tree))

### Simple Split

In [23]:
metrics = {'rmse': mean_squared_error,
           'mae': mean_absolute_error,
           'mape': mean_absolute_percentage_error,
           'msle': mean_squared_log_error}

In [24]:
for pipeline, hyperparameters in experiments:
    experiment_manager(task='regression',
                       pipeline=pipeline, X=X, y=y, 
                       runs=5,
                       validation='simple_split',
                       hyperparameters=hyperparameters,
                       metrics=metrics,
                       random_state=RANDOM_STATE,
                       test_size=TEST_SIZE)

Experiment Name: Ridge
Experiment_id: 3

Run 1

HYPERPARAMETERS
alpha: 1.0
max_iter: 4000
solver: auto
random_state: 77

SCORES
train_rmse: 53.544
test_rmse: 54.399
train_mae: 42.68
test_mae: 45.043
train_mape: 0.386
test_mape: 0.378
train_msle: 0.171
test_msle: 0.162

Run 2

HYPERPARAMETERS
alpha: 0.01
max_iter: 3000
solver: auto
random_state: 77

SCORES
train_rmse: 71.581
test_rmse: 70.172
train_mae: 61.366
test_mae: 60.453
train_mape: 0.577
test_mape: 0.597
train_msle: 0.289
test_msle: 0.292

Run 3

HYPERPARAMETERS
alpha: 0.1
max_iter: 9000
solver: svd
random_state: 77

SCORES
train_rmse: 71.58
test_rmse: 70.171
train_mae: 61.366
test_mae: 60.452
train_mape: 0.577
test_mape: 0.597
train_msle: 0.289
test_msle: 0.292

Run 4

HYPERPARAMETERS
alpha: 0.01
max_iter: 8000
solver: saga
random_state: 77

SCORES
train_rmse: 76.639
test_rmse: 75.524
train_mae: 65.748
test_mae: 64.853
train_mape: 0.62
test_mape: 0.649
train_msle: 0.323
test_msle: 0.332

Run 5

HYPERPARAMETERS
alpha: 100.0
max_i

### Cross Validation

In [25]:
# If the target is transformed, you need to provide the inverse transformation as a function
# using the 'inverse' argument avaivable as a kwargs.

y_log = np.log1p(y)

for pipeline, hyperparameters in experiments:
    experiment_manager(task='regression',
                       pipeline=pipeline, X=X, y=y_log, 
                       runs=5,
                       validation='cross_validation',
                       cv_method=RepeatedKFold(n_splits=2, n_repeats=2, random_state=RANDOM_STATE),
                       hyperparameters=hyperparameters,
                       metrics=metrics,
                       inverse=np.expm1)

Experiment Name: Ridge
Experiment_id: 3

Run 1

HYPERPARAMETERS
alpha: 0.01
max_iter: 3000
solver: auto
random_state: 77

SCORES
train_rmse: 75.258
test_rmse: 75.792
train_mae: 61.682
test_mae: 62.295
train_mape: 0.501
test_mape: 0.508
train_msle: 0.271
test_msle: 0.276

Run 2

HYPERPARAMETERS
alpha: 0.1
max_iter: 4000
solver: cholesky
random_state: 77

SCORES
train_rmse: 75.256
test_rmse: 75.791
train_mae: 61.681
test_mae: 62.294
train_mape: 0.501
test_mape: 0.508
train_msle: 0.271
test_msle: 0.276

Run 3

HYPERPARAMETERS
alpha: 1.0
max_iter: 5000
solver: cholesky
random_state: 77

SCORES
train_rmse: 62.641
test_rmse: 63.428
train_mae: 50.902
test_mae: 51.795
train_mape: 0.408
test_mape: 0.417
train_msle: 0.192
test_msle: 0.198

Run 4

HYPERPARAMETERS
alpha: 0.01
max_iter: 10000
solver: svd
random_state: 77

SCORES
train_rmse: 54.821
test_rmse: 56.64
train_mae: 43.511
test_mae: 44.975
train_mape: 0.35
test_mape: 0.362
train_msle: 0.156
test_msle: 0.166

Run 5

HYPERPARAMETERS
alpha: 0